In [14]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import multiprocessing as mp
from functools import partial
import inspect
import time

In [18]:
df = pd.read_csv('../../Data/yellow_tripdata_2009-01.csv')
zip = pd.read_csv('../../Data/NYC_Zip_Lat_Lon')

In [19]:
df=df.head(10000)

In [20]:
df

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,NaN,NaN,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.00,9.40
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,NaN,NaN,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.00,14.60
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,NaN,NaN,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.00,28.44
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,NaN,NaN,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.00,18.45
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,NaN,NaN,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.00,3.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,VTS,2009-01-03 21:33:00,2009-01-03 21:39:00,1,1.10,-73.997167,40.737425,NaN,NaN,-73.990877,40.749973,CASH,5.3,0.5,NaN,0.00,0.00,5.80
9996,VTS,2009-01-03 19:04:00,2009-01-03 19:19:00,5,2.64,-73.962212,40.779117,NaN,NaN,-73.989327,40.756197,CASH,10.1,0.0,NaN,0.00,0.00,10.10
9997,VTS,2009-01-03 09:54:00,2009-01-03 10:17:00,2,9.06,-73.873022,40.774053,NaN,NaN,-73.971993,40.796870,CASH,24.1,0.0,NaN,0.00,4.15,28.25
9998,VTS,2009-01-07 19:56:00,2009-01-07 20:04:00,1,3.02,-73.984955,40.756653,NaN,NaN,-73.997372,40.721703,Credit,11.3,1.0,NaN,2.00,0.00,14.30


In [9]:
zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    178 non-null    int64  
 1   Borough       178 non-null    object 
 2   Neighborhood  178 non-null    object 
 3   ZipCode       178 non-null    int64  
 4   LAT           177 non-null    float64
 5   LNG           177 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 8.5+ KB


In [10]:
def dist(lat1, long1, lat2, long2):

    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [11]:
def find_nearest(lat, long):
    distances = zip.apply(
        lambda row: dist(lat, long, row['LAT'], row['LNG']), 
        axis=1)
    return zip.loc[distances.idxmin(), 'ZipCode']

In [21]:
start_time = time.time()

df['start_zip'] = df.apply(
    lambda row: find_nearest(row['Start_Lat'], row['Start_Lon']), 
    axis=1)
df.head()

print("--- %s seconds ---" % (time.time() - start_time))

--- 37.95516324043274 seconds ---


885.61 minutes for half necessary metric full query
73 minutes if multicore works for half query
147.6 per sheet
24 sheets minimum
3,542.46 full hypothetical runtime
60 hours (could be done by early next week?)
additional computer for processing is available

In [22]:
df

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt,start_zip
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,NaN,NaN,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.00,9.40,10012
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,NaN,NaN,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.00,14.60,10010
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,NaN,NaN,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.00,28.44,10011
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,NaN,NaN,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.00,18.45,10024
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,NaN,NaN,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.00,3.70,10013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,VTS,2009-01-03 21:33:00,2009-01-03 21:39:00,1,1.10,-73.997167,40.737425,NaN,NaN,-73.990877,40.749973,CASH,5.3,0.5,NaN,0.00,0.00,5.80,10011
9996,VTS,2009-01-03 19:04:00,2009-01-03 19:19:00,5,2.64,-73.962212,40.779117,NaN,NaN,-73.989327,40.756197,CASH,10.1,0.0,NaN,0.00,0.00,10.10,10028
9997,VTS,2009-01-03 09:54:00,2009-01-03 10:17:00,2,9.06,-73.873022,40.774053,NaN,NaN,-73.971993,40.796870,CASH,24.1,0.0,NaN,0.00,4.15,28.25,11369
9998,VTS,2009-01-07 19:56:00,2009-01-07 20:04:00,1,3.02,-73.984955,40.756653,NaN,NaN,-73.997372,40.721703,Credit,11.3,1.0,NaN,2.00,0.00,14.30,10036


In [25]:
df2=df[['Trip_Distance','Fare_Amt','Tip_Amt','Tolls_Amt','start_zip']]

In [26]:
df2

,Trip_Distance,Fare_Amt,Tip_Amt,Tolls_Amt,start_zip
0,2.63,8.9,0.00,0.00,10012
1,4.55,12.1,2.00,0.00,10010
2,10.35,23.7,4.74,0.00,10011
3,5.00,14.9,3.05,0.00,10024
4,0.40,3.7,0.00,0.00,10013
...,...,...,...,...,...
9995,1.10,5.3,0.00,0.00,10011
9996,2.64,10.1,0.00,0.00,10028
9997,9.06,24.1,0.00,4.15,11369
9998,3.02,11.3,2.00,0.00,10036


In [30]:
df3=df2.groupby(['start_zip']).describe()

In [2]:
df3

NameError: name 'df3' is not defined

In [1]:
df3

NameError: name 'df3' is not defined

40.04 MB end goal